<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       ColumnTransformer Function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>The ColumnTransformer function transforms the input data columns in a single operation. Provides only the FIT dataframes generated by the Fit analytic functions, and runs all transformations required in a single operation.
The function performs the following transformations:<ul style = 'font-size:16px;font-family:Arial'>
    <li>Scale Transform</li>
    <li>Bincode Transform</li>
  <li>Function Transform          </li>
<li>NonLinearCombine Transform  </li>
<li>OutlierFilter Transform     </li>
<li>PolynomialFeatures Transform</li>
<li>RowNormalize Transform      </li>
<li>OrdinalEncoding Transform   </li>
<li>OneHotEncoding Transform    </li>
<li>SimpleImpute Transform      </li>
    </ul>
    <p style = 'font-size:16px;font-family:Arial'>
A user must create the FIT dataframe before using the function and the data must also be provided in the same order as in the training data sequence. The FIT dataframe can have maximum of 128 columns.
         <br> In this notebook we will see how we can use the ColumnTransformer function available in Vantage.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import *

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_ColumnTransformer_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_TitanicSurvival_cloud');"        # Takes 30 seconds
#%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_TitanicSurvival_local');" 

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
tdf = DataFrame(in_schema("DEMO_TitanicSurvival", "Passenger_Data"))
print("Shape of the data: ", tdf.shape)
tdf

<p style = 'font-size:16px;font-family:Arial'>Let us create Bin-encoding for age column and One-Hot encoding for sex column</p>

In [ ]:
#Perform Bincode transformation and OneHotEncoding transformation using fit functions and
#             ColumnTransformer() function.
bin_code = BincodeFit(data=tdf,
                      target_columns='age',
                      method_type='Equal-Width',
                      nbins=2,
                      label_prefix='label_prefix')

one_hot_encoding = OneHotEncodingFit(data=tdf,
                                     is_input_dense=True,
                                     target_column="sex",
                                     categorical_values=["male", "female"],
                                     other_column="other")

<p style = 'font-size:16px;font-family:Arial'>We can transform the data from above fit tables in single step using ColumnTransformer function. Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(ColumnTransformer)

In [ ]:
ColumnTransformer_out = ColumnTransformer(fillrowid_column_name="output_value",
                                          input_data=tdf,
                                          bincode_fit_data=bin_code.output,
                                          onehotencoding_fit_data=one_hot_encoding.result)

# Print the result DataFrame.
ColumnTransformer_out.result

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. Cleanup</b>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_TitanicSurvival');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>ColumnTransformer function reference: <a href = 'https://docs.teradata.com/search/all?query=ColumnTransformer&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>